# Dataloaders for CUB Birds, Stanford Dogs, Foodx dataset

In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

## CUB Birds Dataset

In [4]:
class CUBDataset(torchvision.datasets.ImageFolder):
    """
    Dataset class for CUB Dataset
    """

    def __init__(self, image_root_path, caption_root_path=None, split="train", *args, **kwargs):
        """
        Args:
            image_root_path:      path to dir containing images and lists folders
            caption_root_path:    path to dir containing captions
            split:          train / test
            *args:
            **kwargs:
        """
        image_info = self.get_file_content(f"{image_root_path}/images.txt")
        self.image_id_to_name = {y[0]: y[1] for y in [x.strip().split(" ") for x in image_info]}
        split_info = self.get_file_content(f"{image_root_path}/train_test_split.txt")
        self.split_info = {self.image_id_to_name[y[0]]: y[1] for y in [x.strip().split(" ") for x in split_info]}
        self.split = "1" if split == "train" else "0"
        self.caption_root_path = caption_root_path

        super(CUBDataset, self).__init__(root=f"{image_root_path}/images", is_valid_file=self.is_valid_file,
                                         *args, **kwargs)

    def is_valid_file(self, x):
        return self.split_info[(x[len(self.root) + 1:])] == self.split

    @staticmethod
    def get_file_content(file_path):
        with open(file_path) as fo:
            content = fo.readlines()
        return content

In [5]:
data_root = "/home/u20020019/TransFG/CUB_200_2011"

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)


# write data transform here as per the requirement
data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

train_dataset = CUBDataset(image_root_path=f"{data_root}", transform=data_transform, split="train")
test_dataset = CUBDataset(image_root_path=f"{data_root}", transform=data_transform, split="test")


# load in into the torch dataloader to get variable batch size, shuffle 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, drop_last=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, drop_last=False, shuffle=True)

### Use the dataloader

In [6]:
len(train_dataset), len(test_dataset)

(5994, 5794)

In [7]:
len(train_loader), len(test_loader)

(187, 182)

In [7]:
for i, (inputs, labels) in enumerate(train_loader):
    print(inputs.shape)
    print(labels)
    print('='*50)
    break

torch.Size([32, 3, 224, 224])
tensor([ 19,  72,  89,  50, 104, 166,  99, 197, 140, 114, 117, 122,  66, 193,
        176, 136, 191, 155, 101,  36, 171, 102,  45,  16,  99,   2, 150, 130,
         92, 157, 121, 104])


## Stanford Dog Dataset

In [8]:
import torchvision
import scipy.io

In [9]:
class DOGDataset(torchvision.datasets.ImageFolder):
    """
    Dataset class for DOG Dataset
    """

    def __init__(self, image_root_path, caption_root_path=None, split="train", *args, **kwargs):
        """
        Args:
            image_root_path:      path to dir containing images and lists folders
            caption_root_path:    path to dir containing captions
            split:          train / test
            *args:
            **kwargs:
        """
        image_info = self.get_file_content(f"{image_root_path}splits/file_list.mat")
        image_files = [o[0][0] for o in image_info]
        
        split_info = self.get_file_content(f"{image_root_path}/splits/{split}_list.mat")
        split_files = [o[0][0] for o in split_info]
        self.split_info = {}
        if split == 'train' :
            for image in image_files:
                if image in split_files:
                    self.split_info[image] = "1"
                else:
                    self.split_info[image] = "0"
        elif split== 'test' :
            for image in image_files:
                if image in split_files:
                    self.split_info[image] = "0"
                else:
                    self.split_info[image] = "1"
                    
        self.split = "1" if split == "train" else "0"
        self.caption_root_path = caption_root_path

        super(DOGDataset, self).__init__(root=f"{image_root_path}Images", is_valid_file = self.is_valid_file,
                                         *args, **kwargs)

    def is_valid_file(self, x):
        return self.split_info[(x[len(self.root) + 1:])] == self.split

    @staticmethod
    def get_file_content(file_path):
        content =  scipy.io.loadmat(file_path)
        return content['file_list']

In [38]:
# Set train and test set
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


data_root = "/home/u20020019/Fall 2021/CV703 Lab/Week5/dog/"


train_dataset = DOGDataset(image_root_path=f"{data_root}", transform=data_transform, split="train")
test_dataset = DOGDataset(image_root_path=f"{data_root}", transform=data_transform, split="test")
print('Number of train samples:', len(train_dataset))
print('Number of test samples:', len(test_dataset))

# load in into the torch dataloader to get variable batch size, shuffle 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, drop_last=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, drop_last=False, shuffle=True)

Number of train samples: 12000
Number of test samples: 8580


### Use Dataloader

In [11]:
len(train_dataset), len(test_dataset)

(12000, 8580)

In [12]:
len(train_loader), len(test_loader)

(375, 269)

In [13]:
for i, (inputs, labels) in enumerate(test_loader):
    print(inputs.shape)
    print(labels)
    print('='*50)
    break

torch.Size([32, 3, 224, 224])
tensor([ 39,  20,  40,  42,  60,  26,   2,  79,  77, 119, 103,  19,  40, 102,
         50,   8,  73,  55,  63, 110,  66, 113,  42,  84, 102,  32,  98,  78,
         96, 115,  41,  14])


## Food Dataset

In [14]:
import torch
import torchvision
from PIL import Image
import pandas as pd

In [19]:
data_dir = "/home/u20020019/Fall 2021/CV703 Lab/Week5/food_dataset"

split = 'train'
train_df = pd.read_csv(f'{data_dir}/{split}_labels.csv', names= ['image_name','label'])
train_df['path'] = train_df['image_name'].map(lambda x: os.path.join(f'{data_dir}/{split}_set/', x))
train_df

,image_name,label,path
0,img_name,label,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...
1,train_101733.jpg,211,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...
2,train_101734.jpg,211,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...
3,train_101735.jpg,211,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...
4,train_101736.jpg,211,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...
...,...,...,...
118471,train_101728.jpg,123,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...
118472,train_101729.jpg,123,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...
118473,train_101730.jpg,123,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...
118474,train_101731.jpg,123,/home/u20020019/Fall 2021/CV703 Lab/Week5/food...


In [24]:
split = 'val'
val_df = pd.read_csv(f'{data_dir}/{split}_labels.csv', names= ['image_name','label'])
val_df['path'] = val_df['image_name'].map(lambda x: os.path.join(f'{data_dir}/{split}_set/', x))



In [25]:
class FOODDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        return (
            torchvision.transforms.functional.to_tensor(Image.open(row["path"])), row['label']
        )

In [26]:
train_dataset = FOODDataset(train_df)
val_dataset = FOODDataset(val_df)

# load in into the torch dataloader to get variable batch size, shuffle 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, drop_last=True, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, drop_last=False, shuffle=True)

In [27]:
len(train_dataset), len(val_dataset)

(118476, 11995)

In [28]:
len(train_loader), len(val_loader)

(3702, 375)

In [29]:
for i, (inputs, labels) in enumerate(test_loader):
    print(inputs.shape)
    print(labels)
    print('='*50)
    break

torch.Size([32, 3, 224, 224])
tensor([ 35,  39,  46,  24,  88,   3,  16, 105,  57,  98,  32,  79,  43,  19,
         57,  81,  59,  77,  44,  95,   6,  15,  39,  21,  12,  78, 102, 102,
        102,  92, 104, 107])


## Concatenate CUB Birds and Stanford Dogs Dataset

In [30]:
data_root = "/home/u20020019/TransFG/CUB_200_2011"
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)


# write data transform here as per the requirement
data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

train_dataset_cub = CUBDataset(image_root_path=f"{data_root}", transform=data_transform, split="train")
test_dataset_cub = CUBDataset(image_root_path=f"{data_root}", transform=data_transform, split="test")


In [37]:
# Set train and test set
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


data_root = "/home/u20020019/Fall 2021/CV703 Lab/Week5/dog/"

train_dataset_dog = DOGDataset(image_root_path=f"{data_root}", transform=data_transform, split="train")
test_dataset_dog = DOGDataset(image_root_path=f"{data_root}", transform=data_transform, split="test")
print('Number of train samples:', len(train_dataset_dog))
print('Number of test samples:', len(test_dataset_dog))


Number of train samples: 12000
Number of test samples: 8580


In [32]:
# concatenated dataloader for CUB and DOG

train_dataloader = torch.utils.data.DataLoader(
             torch.utils.data.ConcatDataset([train_dataset_cub, train_dataset_dog]),
             batch_size=1, shuffle=True,
             num_workers=1, pin_memory=True)

test_dataloader = torch.utils.data.DataLoader(
             torch.utils.data.ConcatDataset([test_dataset_cub, test_dataset_dog]),
             batch_size=1, shuffle=True,
             num_workers=1, pin_memory=True)

In [33]:
len(train_dataset_cub), len(train_dataset_dog), len(train_dataloader)

(5994, 12000, 17994)

In [34]:
len(test_dataset_cub), len(test_dataset_dog), len(test_dataloader)

(5794, 8580, 14374)

### Iterate Concatenated dataloader images

In [35]:
for i, (inputs, targets) in enumerate(train_dataloader):

    print('image :: ', inputs.shape)
    print(targets)
    break
        

image ::  torch.Size([1, 3, 224, 224])
tensor([46])
